## Input File Compression

### 1. Run-length Compression (4 bits for Run and 4 bits for length)

In [27]:
origin_w = []
origin_act = []

with open("../poster/original_weights.txt", 'r') as act_file:
    for line in act_file:
        line = line.strip()  # Remove leading/trailing spaces
        for i in range(0, len(line), 4):
            bin = line[i:i + 4]  # Extract a 4-bit number
            origin_w.append(int(bin, 2))

with open("../poster/original_activations.txt", 'r') as w_file:
    for line in w_file:
        line = line.strip()  # Remove leading/trailing spaces
        for i in range(0, len(line), 4):
            bin = line[i:i + 4]  # Extract a 4-bit number
            origin_act.append(int(bin, 2))

In [28]:
# Initialize an empty compressed binary string
compressed_binary = ""

# Initialize variables to keep track of the current run length and level
current_run_length = 0
current_level = None

# Iterate through the elements in the list
for value in origin_w:
    if value == 0:
        if current_run_length == 15:
            compressed_binary += format(current_run_length, '04b')  # 4 bits for run length
            compressed_binary += format(value, '04b')  # 4 bits for level
            current_run_length = 0
        # If the value is 0, increase the run length
        else:
            current_run_length += 1
    else:
        # If the value is non-zero, append the run length and level as 4-bit binary strings
        compressed_binary += format(current_run_length, '04b')  # 4 bits for run length
        compressed_binary += format(value, '04b')  # 4 bits for level
        # Reset the run length
        current_run_length = 0

if current_run_length == 0:
    compressed_binary += "0"  # Add the termination bit at the end
else:
    # Add the final run length and level to the compressed binary string
    compressed_binary += format(current_run_length, '04b')  # 4 bits for run length
    compressed_binary += "0000"  # Assuming a non-zero level of 0 
    compressed_binary += "0"  # Add the termination bit at the end

file = open('RLC_weights.txt', 'w') #write to file
file.write(compressed_binary)    
file.close()


In [29]:
# Initialize an empty compressed binary string
compressed_binary = ""

# Initialize variables to keep track of the current run length and level
current_run_length = 0
current_level = None

# Iterate through the elements in the list
for value in origin_act:
    if value == 0:
        if current_run_length == 15:
            compressed_binary += format(current_run_length, '04b')  # 4 bits for run length
            compressed_binary += format(value, '04b')  # 4 bits for level
            current_run_length = 0
        # If the value is 0, increase the run length
        else:
            current_run_length += 1
    else:
        # If the value is non-zero, append the run length and level as 4-bit binary strings
        compressed_binary += format(current_run_length, '04b')  # 4 bits for run length
        compressed_binary += format(value, '04b')  # 4 bits for level
        # Reset the run length
        current_run_length = 0

if current_run_length == 0:
    compressed_binary += "0"  # Add the termination bit at the end
else:
    # Add the final run length and level to the compressed binary string
    compressed_binary += format(current_run_length, '04b')  # 4 bits for run length
    compressed_binary += "0000"  # Assuming a non-zero level of 0 
    compressed_binary += "0"  # Add the termination bit at the end

file = open('RLC_activations.txt', 'w') #write to file
file.write(compressed_binary)    
file.close()


### 2. Compressed Sparse Column (CSC) format

In [2]:
origin_w = []
origin_act = []

with open("../poster/original_weights.txt", 'r') as act_file:
    for line in act_file:
        temp = []
        line = line.strip()  # Remove leading/trailing spaces
        for i in range(0, len(line), 4):
            bin = line[i:i + 4]  # Extract a 4-bit number
            temp.append(int(bin, 2))
        origin_w.append(temp)

with open("../poster/original_activations.txt", 'r') as w_file:
    for line in w_file:
        temp = []
        line = line.strip()  # Remove leading/trailing spaces
        for i in range(0, len(line), 4):
            bin = line[i:i + 4]  # Extract a 4-bit number
            temp.append(int(bin, 2))
        origin_act.append(temp)

In [41]:
import numpy as np

origin_w = np.array(origin_w)

IP_list = np.array([0])
RI_list = np.array([])
data = np.array([])

for i in range(len(origin_w[0])):
    IP = IP_list[len(IP_list)-1] + np.sum(origin_w[:,i] != 0)
    IP_list = np.append(IP_list,IP)

    RI = np.nonzero(origin_w[:,i])[0]
    if RI_list.size == 0:
        RI_list = RI
    elif RI.size != 0:
        RI_list = np.hstack((RI_list, RI))

    datum = origin_w[:,i][RI]
    if data.size == 0:
        data = datum
    elif datum.size != 0:
        data = np.hstack((data, datum))

In [62]:
IP_str = ""
RI_str = ""
data_str = ""

for d in IP_list:
    IP_str = IP_str + '{0:08b}'.format(d)

for d in RI_list:
    RI_str = RI_str + '{0:08b}'.format(d)   

for d in data:
    data_str = data_str + '{0:08b}'.format(d)

In [63]:
file = open('CSC_weights.txt', 'w') #write to file
file.write(IP_str)    
file.write(RI_str)    
file.write(data_str)    
file.close()

In [64]:
file = open('CSE_length.txt', 'w') #write to file
file.write('{0:008b}'.format(len(IP_str)))    
file.write('{0:008b}'.format(len(RI_str)))    
file.write('{0:008b}'.format(len(data_str)))  
file.close()

In [3]:
import numpy as np

origin_act = np.array(origin_act)

IP_list = np.array([0])
RI_list = np.array([])
data = np.array([])

for i in range(len(origin_act[0])):
    IP = IP_list[len(IP_list)-1] + np.sum(origin_act[:,i] != 0)
    IP_list = np.append(IP_list,IP)

    RI = np.nonzero(origin_act[:,i])[0]
    if RI_list.size == 0:
        RI_list = RI
    elif RI.size != 0:
        RI_list = np.hstack((RI_list, RI))

    datum = origin_act[:,i][RI]
    if data.size == 0:
        data = datum
    elif datum.size != 0:
        data = np.hstack((data, datum))

In [4]:
IP_str = ""
RI_str = ""
data_str = ""

for d in IP_list:
    IP_str = IP_str + '{0:08b}'.format(d)

for d in RI_list:
    RI_str = RI_str + '{0:08b}'.format(d)   

for d in data:
    data_str = data_str + '{0:08b}'.format(d)

In [5]:
file = open('CSC_activations.txt', 'w') #write to file
file.write(IP_str)    
file.write(RI_str)    
file.write(data_str)    
file.close()

In [6]:
file = open('CSC_length_act.txt', 'w') #write to file
file.write('{0:008b}'.format(len(IP_str)))    
file.write('{0:008b}'.format(len(RI_str)))    
file.write('{0:008b}'.format(len(data_str)))  
file.close()